# Refreshing tool definitions of your MCP servers in AgentCore Gateway

## Overview

The synchronization mechanism in AgentCore Gateway ensures accurate tool definitions from MCP server targets. It manages schema consistency, optimizes performance, and maintains data integrity through both explicit and implicit synchronization methods.

### Explicit Synchronization

The SynchronizeGateway API is an asynchronous operation that enables on-demand synchronization of tools from MCP server targets. This API provides customers with explicit control over when to refresh their tool definitions, particularly useful after making changes to their MCP server's tool configurations. For targets configured with OAuth authentication, the API first validates credentials through the AgentCore Identity service before proceeding with MCP server communication. If validation fails, the synchronization operation fails with appropriate error details, transitioning the target to a FAILED state. For targets configured without authentication, the API proceeds directly to tool synchronization.

The tool processing workflow begins with establishing a session with the MCP server. The API then retrieves and processes tools in optimized batches, adding target-specific prefixes to prevent naming collisions with tools from other targets. The API enforces a limit of 10,000 tools per target and ensures tool definitions are normalized while preserving essential metadata from the original MCP server definitions.

In this tutorial, we will update the MCP Server created in '01-Gateway-MCP-Target' by adding additional tools and then invoke the SynchronizeGateway API to explicitly synchronize the latest tool definitions.

![Diagram](images/mcp-server-target-explicit-sync.png)

### Implicit Synchronization

AgentCore Gateway performs an implicit synchronization during CreateGatewayTarget and UpdateGatewayTarget operations that differs from the explicit SynchronizeGateway API. This built-in synchronization ensures that newly created or updated MCP targets are immediately usable while maintaining data consistency. This implicit synchronization ensures that MCP targets are always created or updated with valid, current tool definitions, maintaining Gateway's guarantee that any target in READY state is immediately usable. 

In this tutorial, we will create a new MCP Server and call UpdateGatewayTarget operation to update the new MCP target that implicitly synchronize the tool definitions from new MCP Server.

![Diagram](images/mcp-server-target-implicit-sync.png)

### Tutorial Details


| Information          | Details                                                              |
|:---------------------|:---------------------------------------------------------------------|
| Tutorial type        | Interactive                                                          |
| AgentCore components | AgentCore Gateway, AgentCore Identity, AgentCore Runtime             |
| Agentic Framework    | Strands Agents                                                       |
| Gateway Target type  | MCP server                                                           |
| Agent                | Strands                                                              |
| Inbound Auth IdP     | Amazon Cognito, but can use others                                   |
| Outbound Auth        | Amazon Cognito, but can use others                                   |
| LLM model            | Anthropic Claude Sonnet 4                                            |
| Tutorial components  | Creating AgentCore Gateway with MCP Target and synchronize the tools |
| Tutorial vertical    | Cross-vertical                                                       |
| Example complexity   | Easy                                                                 |
| SDK used             | boto3                                                                |

## Tutorial architecture

In this tutorial, we will describe how to trigger both explicit and implicit synchronization of tool definitions from MCP server targets within AgentCore Gateway

## Prerequisites

To execute this tutorial you will need:
* Jupyter notebook (Python kernel)
* uv
* AWS credentials
* Amazon Cognito

In [ ]:
# Install from the requirements file or pyproject.toml file in current directory
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
# Import utils
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

# Setup logging 
import logging

# Configure logging for notebook environment
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# Set specific logger levels
logging.getLogger("gateway").setLevel(logging.INFO)

# Perform Explicit Synchronization

## Retrieve the Cognito User Pool Information for Inbound authorization to Gateway

In [ ]:
# Creating Cognito User Pool 
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "invoke",  # Just 'invoke', will be formatted as resource_server_id/invoke
    "ScopeDescription": "Scope for invoking the agentcore gateway"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
scopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
gw_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {gw_user_pool_id}")

utils.get_or_create_resource_server(cognito, gw_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

gw_client_id, gw_client_secret = utils.get_or_create_m2m_client(cognito, gw_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"Client ID: {gw_client_id}")

# Get discovery URL
gw_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/openid-configuration'
print(gw_cognito_discovery_url)

## Retrieve the Cognito User Pool Information for Inbound authorization to Runtime

In [ ]:
# Creating Cognito User Pool
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-runtime-pool"
RESOURCE_SERVER_ID = "sample-agentcore-runtime-id"
RESOURCE_SERVER_NAME = "sample-agentcore-runtime-name"
CLIENT_NAME = "sample-agentcore-runtime-client"
SCOPES = [
    {"ScopeName": "invoke",  # Just 'invoke', will be formatted as resource_server_id/invoke
    "ScopeDescription": "Scope for invoking the agentcore gateway"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
runtimeScopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
runtime_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {runtime_user_pool_id}")

utils.get_or_create_resource_server(cognito, runtime_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

runtime_client_id, runtime_client_secret = utils.get_or_create_m2m_client(cognito, runtime_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"Client ID: {runtime_client_id}")

# Get discovery URL
runtime_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{runtime_user_pool_id}/.well-known/openid-configuration'
print(runtime_cognito_discovery_url)

## Add a new tool to the existing MCP Server

Add a new tool 'cancelOrder' to the existing MCP Server you created in '01-mcp-server-target' tutorial.

In [ ]:
%%writefile mcp_server_updated.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def getOrder() -> int:
    """Get an order"""
    return 123

@mcp.tool()
def updateOrder(orderId: int) -> int:
    """Update existing order"""
    return 456

@mcp.tool()
def cancelOrder(orderId: int) -> int:
    """cancel existing order"""
    return 789

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Next, we will configure the AgentCore Runtime to deploy the updated MCP server with the new 'cancelOrder' tool.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server_updated.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")
agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            runtime_client_id
        ],
        "discoveryUrl": runtime_cognito_discovery_url
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server_updated.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="ac_gateway_mcp_server"
)
print("Configuration completed ✓")

Then, redeploy the updated MCP Server to AgentCore Runtime.

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()

agent_arn = launch_result.agent_arn
agent_id = launch_result.agent_id

encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')

agent_url = f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'
print("Launch completed ✓")
print(f"Agent ARN: {agent_arn}")
print(f"Agent ID: {agent_id}")

## Invoke SynchronizeGateway API

### Retrieve Outbound Auth provider

Retrieve the existing OAuth2 credential provider we created in previous notebook

In [ ]:
identity_client = boto3.client('bedrock-agentcore-control', region_name=region)

# Get credential provider 
response = identity_client.get_oauth2_credential_provider(name="ac-gateway-mcp-server-identity")
cognito_provider_arn = response['credentialProviderArn']
print(cognito_provider_arn)

### Check the current tool definitions

Let's invoke a strands agent to list MCP tools using Bedrock AgentCore Gateway. This will return the old tool list (without 'cancelOrder'), because we have not yet asked the Gateway to synchronize.

Retrieve the gateway identifier we created in previous notebook.

In [ ]:
gateway_name = 'ac-gateway-mcp-server'

gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
list_gateways_response = gateway_client.list_gateways()

gatewayID = next(
    g['gatewayId'] for g in list_gateways_response['items'] 
    if g['name'] == gateway_name
)

print(gatewayID)

Retrieve the gateway URL.

In [ ]:
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
gateway_response = gateway_client.get_gateway(gatewayIdentifier=gatewayID)

gatewayURL = gateway_response['gatewayUrl']

print(gatewayURL)

In [ ]:
import json

import requests
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent


def get_token():
    token = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
    return token['access_token']


def create_streamable_http_transport():
    return streamablehttp_client(
        gatewayURL, headers={"Authorization": f"Bearer {get_token()}"}
    )


client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0", # may need to update model_id depending on region
    temperature=0.7,
    max_tokens=500,  # Limit response length
)

with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## you can replace with any model you like
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi, can you list all tools available to you")  

### Call SynchronizeGateway API 

Retrieve the MCP Target Id

In [ ]:
gateway_target_name = 'mcp-server-target'

gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
list_targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gatewayID)

gatewayTargetID = next(
    g['targetId'] for g in list_targets_response['items'] 
    if g['name'] == gateway_target_name
)

print(gatewayTargetID)

Call the Synchronize Gateway API to perform explicit synchronization

In [ ]:
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

synchronize_gateway_response = gateway_client.synchronize_gateway_targets(
    gatewayIdentifier=gatewayID,
    targetIdList=[gatewayTargetID]
    )

print(synchronize_gateway_response)

### Check the current tool definitions after the synchronization

Let's invoke a strands agent to list MCP tools using Bedrock AgentCore Gateway. This will return the new tool list.

In [ ]:
with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## you can replace with any model you like
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi, can you list all tools available to you")   

# Perform Implicit Synchronization

## Add another new tool to the existing MCP Server

Add a new tool 'deleteOrder' to the existing MCP Server.

In [ ]:
%%writefile mcp_server_updated.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def getOrder() -> int:
    """Get an order"""
    return 123

@mcp.tool()
def updateOrder(orderId: int) -> int:
    """Update existing order"""
    return 456

@mcp.tool()
def cancelOrder(orderId: int) -> int:
    """cancel existing order"""
    return 789

@mcp.tool()
def deleteOrder(orderId: int) -> int:
    """delete existing order"""
    return 101
    
if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Next, we will configure the AgentCore Runtime to deploy the updated MCP server with the new 'deleteOrder' tool.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server_updated.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")
agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            runtime_client_id
        ],
        "discoveryUrl": runtime_cognito_discovery_url
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server_updated.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="ac_gateway_mcp_server"
)
print("Configuration completed ✓")

Then, redeploy the updated MCP Server to AgentCore Runtime.

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()

agent_arn = launch_result.agent_arn
agent_id = launch_result.agent_id

encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')

agent_url = f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'
print("Launch completed ✓")
print(f"New Agent ARN: {agent_arn}")
print(f"New Agent ID: {agent_id}")

## Update the existing Gateway Target with MCP Server

This update gateway target operation will update the gateway target with MCP Server (new version) and also perform implicit synchronization to sync the tool definitions

In [ ]:
import boto3

gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
update_gateway_target_response = gateway_client.update_gateway_target(
    gatewayIdentifier=gatewayID,
    targetId = gatewayTargetID,
    name = 'mcp-server-target',
    targetConfiguration={
        'mcp': {
            'mcpServer': {
                'endpoint': agent_url
            }
        }
    },
    credentialProviderConfigurations=[
        {
            'credentialProviderType': 'OAUTH',
            'credentialProvider': {
                'oauthCredentialProvider': {
                    'providerArn': cognito_provider_arn,
                    'scopes': [
                        runtimeScopeString
                    ]
                }
            }
        },
    ]
)

print(update_gateway_target_response)

## Check the current tool definitions after updating the MCP Server target

In [ ]:
with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## you can replace with any model you like
    # Invoke the agent with the sample prompt. This will only invoke MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi, can you list all tools available to you")   

# Clean up
Additional resources are also created like IAM role, IAM Policies, Credentials provider, AWS Lambda functions, Cognito user pools, s3 buckets that you might need to manually delete as part of the clean up. This depends on the example you run.

## Delete the Gateway (Optional)

In [ ]:
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

utils.delete_gateway(gateway_client, gatewayID)

### Delete the Identity Provider (Optional)

In [ ]:
identity_client.delete_oauth2_credential_provider(name='ac-gateway-mcp-server-identity')


#### Delete the Runtime (Optional) 

In [ ]:
import boto3
runtime_client = boto3.client('bedrock-agentcore-control',
                              region_name=REGION)

runtime_client.delete_agent_runtime(agentRuntimeId=agent_id)